In [1]:
import findspark
import datetime as dt

findspark.init('C:\spark-3.3.2-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession, Row
from pyspark import SparkContext

In [3]:
ss = SparkSession.builder.appName('aggs').getOrCreate()

In [4]:
type(ss)

pyspark.sql.session.SparkSession

In [7]:
sdf = ss.read.csv("walmart_stock.csv", sep=",", inferSchema="true", header="true")

In [8]:
sdf.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [13]:
sdf.show(5)

+-------------------+------------------+---------+---------+------------------+--------+------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



## scenario 1:  
### print out first 5 columns ?

In [11]:
descr = sdf.describe()

## scenario2:
### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to ust show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. 


In [83]:
descr.show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

In [14]:
descr.createOrReplaceTempView("describe")

In [86]:
ss.sql("""
select  
summary,
round(Open, 2) Open, 
round(High, 2) High, 
round(Low, 2) Low, 
round(Close, 2) Close, 
round(Volume, 2) Volume,
round(`Adj Close`,2) Adj_Close
from describe
""").show()

+-------+------+------+------+------+----------+---------+
|summary|  Open|  High|   Low| Close|    Volume|Adj_Close|
+-------+------+------+------+------+----------+---------+
|  count|1258.0|1258.0|1258.0|1258.0|    1258.0|   1258.0|
|   mean| 72.36| 72.84| 71.92| 72.39|8222093.48|    67.24|
| stddev|  6.77|  6.77|  6.74|  6.76|4519780.84|     6.72|
|    min| 56.39| 57.06|  56.3| 56.42| 2094900.0|    50.36|
|    max|  90.8| 90.97| 89.25| 90.47| 8.08981E7|    84.91|
+-------+------+------+------+------+----------+---------+



## scenario3:

### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.?


In [32]:
sdf.createOrReplaceTempView("walmart")

In [34]:
ss.sql("select * from walmart limit 5").show()

+-------------------+------------------+---------+---------+------------------+--------+------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+-------------------+------------------+---------+---------+------------------+--------+------------------+



In [42]:
sdf2 = ss.sql("""
select Date, High, Volume, High/Volume as HV from walmart
""")
sdf2.show(5)


+-------------------+---------+--------+--------------------+
|               Date|     High|  Volume|                  HV|
+-------------------+---------+--------+--------------------+
|2012-01-03 00:00:00|61.060001|12668800|4.819714653321546E-6|
|2012-01-04 00:00:00|60.349998| 9593300|6.290848613094555E-6|
|2012-01-05 00:00:00|59.619999|12768200|4.669412994783916E-6|
|2012-01-06 00:00:00|59.450001| 8069400|7.367338463826307E-6|
|2012-01-09 00:00:00|59.549999| 6679300|8.915604778943901E-6|
+-------------------+---------+--------+--------------------+
only showing top 5 rows



## scenario4: 
### What day had the Peak High in Price?

In [48]:
rdd2 = ss.sql("select * from walmart where High=(select max(High) from walmart)")
rdd2.show()

+-------------------+---------+---------+-----+---------+-------+---------+
|               Date|     Open|     High|  Low|    Close| Volume|Adj Close|
+-------------------+---------+---------+-----+---------+-------+---------+
|2015-01-13 00:00:00|90.800003|90.970001|88.93|89.309998|8215400|83.825448|
+-------------------+---------+---------+-----+---------+-------+---------+



## scenario5:

### What is the mean of the Close column?


In [50]:
rdd3 = ss.sql("select mean(Close) as Close_mean from walmart")
rdd3.show()

+-----------------+
|       Close_mean|
+-----------------+
|72.38844998012726|
+-----------------+



## scenario6:

### What is the max and min of the Volume column?¶


In [51]:
rdd4 = ss.sql("select max(Volume) as Max_volume, min(Volume) as Min_volume from walmart")
rdd4.show()

+----------+----------+
|Max_volume|Min_volume|
+----------+----------+
|  80898100|   2094900|
+----------+----------+



## scenario7:

### How many days was the Close lower than 60 dollars?


In [52]:
rdd5 = ss.sql("select count(*) as Count_Close from walmart where Close<60")
rdd5.show()

+-----------+
|Count_Close|
+-----------+
|         81|
+-----------+



## Scenario8:

### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [56]:
rdd6 = ss.sql("select (select count(*) as count_80 from walmart where High>80)*100/count(*) as percent from walmart")
rdd6.show()

+-----------------+
|          percent|
+-----------------+
|9.141494435612083|
+-----------------+



## Scenario9:

### What is the maximum value of the "High" column for each year in the dataset?

In [59]:
rdd7 = ss.sql("""
select distinct year(Date) as Year,max(High) over(partition by year(Date)) as Max_per_year
from walmart
""")
rdd7.show()

+----+------------+
|Year|Max_per_year|
+----+------------+
|2015|   90.970001|
|2012|   77.599998|
|2014|   88.089996|
|2016|   75.190002|
|2013|   81.370003|
+----+------------+

